# NOTEBOOK TO PREPROCESS THE DATA (then used for the project accomplishment)

## **INFORMATIONS ON THE CSVs**

*Data*: BeerAdvocate / RateBeer / matched_beer_data

*Difference ratings-reviews*: **reviews.txt** appears to be a subset of **ratings.txt** because the latter also has the review column (True or False) and **reviews.txt** is the set of all ratings that are True.

*Code to print .txt*: 
* """with open(BA_REVIEWS_DATASET, 'r', encoding='utf-8') as file:
    for _ in range(16):
        print(file.readline())"""
* """with open(BA_RATINGS_DATASET, 'r', encoding='utf-8') as file:
    for _ in range(17):
        print(file.readline())"""
* !head Data/BeerAdvocate/ratings.txt/ratings.txt
* """from collections import deque
n_last_lines = 10
with open(BA_REVIEWS_DATASET, 'r', encoding='utf-8') as file:
    last_lines = deque(file, maxlen=n_last_lines)
for line in last_lines:
    print(line.strip())"""

### BeerAdvocate

**beers.csv**
* beer_id
* beer_name
* brewery_id
* brewery_name
* style
* nbr_ratings
* nbr_reviews
* avg
* ba_score
* bros_score
* abv
* avg_computed
* zscore
* nbr_matched_valid_ratings
* avg_matched_valid_ratings

**breweries.csv**
* id,
* location
* name
* nbr_beers

**users.csv**
* nbr_ratings
* nbr_reviews
* user_id
* user_name
* joined
* location

**ratings.txt** (line format i.e. Header=None)
* beer_name
* beer_id
* brewery_name
* brewery_id
* style
* abv
* date
* user_name
* user_id
* appearance
* aroma
* palate
* taste
* overall
* rating
* text
* review: *True or False*

**reviews.txt** (line format i.e. Header=None, subset of **ratings.txt**)
* beer_name
* beer_id
* brewery_name
* brewery_id
* style
* abv
* date
* user_name
* user_id
* appearance : *up to 5*
* aroma : *up to 5*
* palate : *up to 5*
* taste : *up to 5*
* overall : *up to 5*
* rating : *up to 5, unkown formula but different weights for each parameter*
* text

----------------------------------------------------------------------------------------------------

### RateBeer

**beers.csv**
* beer_id
* beer_name
* brewery_id
* brewery_name
* style
* nbr_ratings
* overall_score
* style_score
* avg
* abv
* avg_computed
* zscore
* nbr_matched_valid_ratings
* avg_matched_valid_ratings

**breweries.csv**
* id
* location
* name
* nbr_beers

**users.csv**
* nbr_ratings
* user_id
* user_name
* joined
* location

**ratings.txt = reviews.txt** (line format i.e. Header=None)
* beer_name
* beer_id
* brewery_name
* brewery_id
* style
* abv
* date
* user_name
* user_id
* appearance : *up to 5*
* aroma : *up to 10*
* palate (=mouthfeel) : *up to 5*
* taste : *up to 10*
* overall : *up to 20*
* rating : *up to 50 (sum of all previous) then divided by 10 --> up to 5*
* text

----------------------------------------------------------------------------------------------------

### matched_beer_data

**beers.csv**
#### ba:
* abv
* avg
* avg_computed
* avg_matched_valid_ratings
* ba_score
* beer_id
* beer_name
* beer_wout_brewery_name
* brewery_id
* brewery_name
* bros_score
* nbr_matched_valid_ratings
* nbr_ratings
* nbr_reviews
* style
* zscore
#### rb:
* abv
* avg
* avg_computed
* avg_matched_valid_ratings
* beer_id
* beer_name
* beer_wout_brewery_name
* brewery_id
* brewery_name
* nbr_matched_valid_ratings
* nbr_ratings
* overall_score
* style
* style_score
* zscore
#### scores:
* diff
* sim

**breweries.csv**
#### ba:
* id
* location
* name
* nbr_beers
#### rb:
* id
* location
* name
* nbr_beers
#### scores:
* diff
* sim

**ratings.csv**
#### ba:
* abv
* appearance
* aroma
* beer_id
* beer_name
* brewery_id
* brewery_name
* date
* overall
* palate
* rating
* review
* style
* taste
* text
* user_id
* user_name
#### rb:
* abv
* appearance
* aroma
* beer_id
* beer_name
* brewery_id
* brewery_name
* date
* overall
* palate
* rating
* style
* taste
* text
* user_id
* user_name


**users_approx.csv**
#### ba:
* joined
* location
* nbr_ratings
* nbr_reviews
* user_id
* user_name
* user_name_lower
#### rb:
* joined
* location
* nbr_ratings
* user_id
* user_name
* user_name_lower
#### scores:
* sim

**users.csv** (is a subset of **users_approx** --> it is composed of users from **users_approx** where `sim` closed to 1)
#### ba:
* joined
* location
* nbr_ratings
* nbr_reviews
* user_id
* user_name
* user_name_lower
#### rb:
* joined
* location
* nbr_ratings
* user_id
* user_name
* user_name_lower

----------------------------------------------------------------------------------------------------

## **LOADING DATAs**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

python(52575) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [2]:
DATA_FOLDER = '/Users/zoemonnard/Documents/ada-2024-project-bada55/data/beer_dataset/'
BEER_ADVOCATE_FOLDER = DATA_FOLDER + 'BeerAdvocate/' #BA
RATE_BEER_FOLDER = DATA_FOLDER + 'RateBeer/' #RB

BA_REVIEWS_DATASET = BEER_ADVOCATE_FOLDER + "reviews.txt"

RB_REVIEWS_DATASET = RATE_BEER_FOLDER + "reviews.txt"

## **PROCESSING BEERADVOCATE**

### ratings.csv / reviews.csv

ratings.txt != reviews.txt

ratings :
[151 074 576 lines i.e. 151 074 576/18 = 8 393 032 reviews]

reviews :
[4 4022 962 lines i.e. 44 022 962/17 = 2 589 586 reviews]

----------------------------------------------------------------------------------------------------

Treatment of .txt to df

In [3]:
import os
import pandas as pd

columns = ['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date', 
           'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall', 
           'rating', 'text']
chunk_size = 1_000_000
data = []
entry_count = 0
chunk_count = 0
current_entry = {}

output_dir = "../../generated/ba_chunks/"
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

with open(BA_REVIEWS_DATASET, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
            key, value = line.split(':', 1)
            key = key.strip()
            value = value.strip()
            current_entry[key] = value
        else:
            if current_entry:
                data.append(current_entry)
                current_entry = {}
                entry_count += 1

                # Save chunk when reaching chunk size
                if entry_count >= chunk_size:
                    chunk_df = pd.DataFrame(data, columns=columns)
                    chunk_file_path = f"{output_dir}ba_reviews_chunk_{chunk_count}.parquet"
                    chunk_df.to_parquet(chunk_file_path)
                    print(f"Saved {chunk_file_path}")
                    data = []
                    entry_count = 0
                    chunk_count += 1
                    
# Process any remaining entries after the loop
if data:
    chunk_df = pd.DataFrame(data, columns=columns)
    chunk_file_path = f"{output_dir}ba_reviews_chunk_{chunk_count}.parquet"
    chunk_df.to_parquet(chunk_file_path)
    print(f"Saved {chunk_file_path}")

print('Saving done')

Saved ../../generated/ba_chunks/ba_reviews_chunk_0.parquet
Saved ../../generated/ba_chunks/ba_reviews_chunk_1.parquet
Saved ../../generated/ba_chunks/ba_reviews_chunk_2.parquet
Saving done


In [3]:
columns = ['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date', 
           'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall', 
           'rating', 'text']
chunk_size = 1_000_000
data = []
entry_count = 0
chunk_count = 0
current_entry = {}

with open(BA_REVIEWS_DATASET, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()
                current_entry[key] = value
        else:
            if current_entry:
                data.append(current_entry)
                current_entry = {}
                entry_count += 1

                # Save chunk when reaching chunk size
                if entry_count >= chunk_size:
                    chunk_df = pd.DataFrame(data, columns=columns)
                    chunk_file_path = f"../../generated/ba_chunks/ba_reviews_chunk_{chunk_count}.parquet"
                    chunk_df.to_parquet(chunk_file_path)
                    print(f"Saved {chunk_file_path}")
                    data = []
                    entry_count = 0
                    chunk_count += 1
                    
# Process any remaining entries after the loop
if data:
    chunk_df = pd.DataFrame(data, columns=columns)
    chunk_file_path = f"../../generated/ba_chunks/ba_reviews_chunk_{chunk_count}.parquet"
    chunk_df.to_parquet(chunk_file_path)
    print(f"Saved {chunk_file_path}")

print('Saving done')

FileNotFoundError: [Errno 2] No such file or directory: '../../generated/ba_chunks/ba_reviews_chunk_0.parquet'

In [4]:
ba_chunk_files = glob.glob("../../generated/ba_chunks/ba_reviews_chunk_*.parquet")
ba_reviews = pd.concat([pd.read_parquet(ba_chunk) for ba_chunk in ba_chunk_files], ignore_index=True)

In [5]:
empty_text = ba_reviews['text'] == ''
ba_reviews.drop(ba_reviews[empty_text].index, inplace= True)
missing_values = np.where(pd.isnull(ba_reviews))
if missing_values[0].size == 0:
    print("The dataset is clean, with no missing values.")
else:
    print("The dataset has missing values at:")
    print("Row indices:", missing_values[0])
    print("Column indices:", missing_values[1])

The dataset is clean, with no missing values.


In [6]:
cols_to_numeric = ['beer_id', 'brewery_id', 'abv', 'date', 'appearance', 'aroma', 'palate', 'taste', 'overall', 'rating']
ba_reviews[cols_to_numeric] = ba_reviews[cols_to_numeric].apply(pd.to_numeric, errors = 'coerce')
ba_reviews['date'] = pd.to_datetime(ba_reviews['date'], unit='s').dt.strftime('%d/%m/%Y')

In [10]:
absolute_path = os.path.abspath(output_dir)
print("Absolute path to output files:", absolute_path)

Absolute path to output files: /Users/zoemonnard/Documents/generated/ba_chunks


## **PROCESSING RATEBEER**

### ratings.csv / reviews.csv

ratings.txt = reviews.txt (i.e. no difference for this dataset)

[121 075 258 lines i.e. 121075258/17 = 7 122 074 review]

----------------------------------------------------------------------------------------------------

Treatment of .txt to df

In [7]:
columns = ['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date', 
           'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall', 
           'rating', 'text']
chunk_size = 1_000_000
data = []
entry_count = 0
chunk_count = 0
current_entry = {}

with open(RB_REVIEWS_DATASET, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if line:
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()
                current_entry[key] = value
        else:
            if current_entry:
                data.append(current_entry)
                current_entry = {}
                entry_count += 1

                # Save chunk when reaching chunk size
                if entry_count >= chunk_size:
                    chunk_df = pd.DataFrame(data, columns=columns)
                    chunk_file_path = f"../../generated/rb_chunks/rb_reviews_chunk_{chunk_count}.parquet"
                    chunk_df.to_parquet(chunk_file_path)
                    print(f"Saved {chunk_file_path}")
                    data = []
                    entry_count = 0
                    chunk_count += 1
                    
# Process any remaining entries after the loop
if data:
    chunk_df = pd.DataFrame(data, columns=columns)
    chunk_file_path = f"../../generated/rb_chunks/rb_reviews_chunk_{chunk_count}.parquet"
    chunk_df.to_parquet(chunk_file_path)
    print(f"Saved {chunk_file_path}")

print('Saving done')

Saved ../../generated/rb_chunks/rb_reviews_chunk_0.parquet
Saved ../../generated/rb_chunks/rb_reviews_chunk_1.parquet
Saved ../../generated/rb_chunks/rb_reviews_chunk_2.parquet
Saved ../../generated/rb_chunks/rb_reviews_chunk_3.parquet
Saved ../../generated/rb_chunks/rb_reviews_chunk_4.parquet
Saved ../../generated/rb_chunks/rb_reviews_chunk_5.parquet
Saved ../../generated/rb_chunks/rb_reviews_chunk_6.parquet
Saved ../../generated/rb_chunks/rb_reviews_chunk_7.parquet
Saving done


In [8]:
rb_chunk_files = glob.glob("../../generated/rb_chunks/rb_reviews_chunk_*.parquet")
rb_reviews = pd.concat([pd.read_parquet(rb_chunk) for rb_chunk in rb_chunk_files], ignore_index=True)

In [9]:
empty_text = rb_reviews['text'] == ''
rb_reviews.drop(rb_reviews[empty_text].index, inplace= True)
np.where(pd.isnull(rb_reviews))
if missing_values[0].size == 0:
    print("The dataset is clean, with no missing values.")
else:
    print("The dataset has missing values at:")
    print("Row indices:", missing_values[0])
    print("Column indices:", missing_values[1])

The dataset is clean, with no missing values.


In [10]:
cols_to_numeric = ['beer_id', 'brewery_id', 'abv', 'date', 'appearance', 'aroma', 'palate', 'taste', 'overall', 'rating']
rb_reviews[cols_to_numeric] = rb_reviews[cols_to_numeric].apply(pd.to_numeric, errors = 'coerce')
rb_reviews['date'] = pd.to_datetime(rb_reviews['date'], unit='s').dt.strftime('%d/%m/%Y')

## **SAVING PROCESSED DFs**

In [11]:
ba_reviews.to_parquet('../../generated/new_ba_reviews.parquet')
rb_reviews.to_parquet('../../generated/new_rb_reviews.parquet')